In [1]:
from sqlalchemy import create_engine, MetaData

In [2]:
engine = create_engine("sqlite:///arlchmy_core.db", echo =True )

In [3]:
con = engine.connect()

2018-07-31 12:47:44,131 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-31 12:47:44,144 INFO sqlalchemy.engine.base.Engine ()
2018-07-31 12:47:44,148 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-31 12:47:44,153 INFO sqlalchemy.engine.base.Engine ()


In [4]:
metadata = MetaData(bind = engine, reflect= True)

2018-07-31 12:47:55,839 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-31 12:47:55,840 INFO sqlalchemy.engine.base.Engine ()
2018-07-31 12:47:55,846 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-31 12:47:55,847 INFO sqlalchemy.engine.base.Engine ()
2018-07-31 12:47:55,850 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-07-31 12:47:55,851 INFO sqlalchemy.engine.base.Engine ()
2018-07-31 12:47:55,854 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Album")
2018-07-31 12:47:55,855 INFO sqlalchemy.engine.base.Engine ()
2018-07-31 12:47:55,857 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-07-31 12:47:55,858 INFO sqlalchemy.engine.base.Engine ()
2018-07

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  """Entry point for launching an IPython kernel.


In [5]:
metadata.reflect(bind=engine) # 바인딩을 엔진을 주고 메타데이터 가져옴

2018-07-19 00:24:17,112 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-19 00:24:17,113 INFO sqlalchemy.engine.base.Engine ()


In [6]:
for row in metadata.tables: #메모리상 존재하는지 참조
    print(row)

Album
Artist
Genre
track


In [7]:
album = metadata.tables['Album']

In [8]:
result = con.execute(album.select())

2018-07-19 00:24:18,396 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"
2018-07-19 00:24:18,397 INFO sqlalchemy.engine.base.Engine ()


In [9]:
for row in result:
    print(row)

(1, 'IV', 1)
(2, 'who Made Who', 2)


# SQLAlchemy ORM

In [10]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///orm.db", echo = True)
# 중간과정보기 # 엔진 만들기

In [11]:
conn = engine.connect()

2018-07-19 00:24:20,092 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-19 00:24:20,093 INFO sqlalchemy.engine.base.Engine ()
2018-07-19 00:24:20,094 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-19 00:24:20,095 INFO sqlalchemy.engine.base.Engine ()


In [1]:
from sqlalchemy.ext.declarative import declarative_base

In [13]:
base = declarative_base() # metadata와 유사 -> 다른점 클래스를 만듬

In [2]:
from sqlalchemy import Column, Integer, String, ForeignKey

# relationship

In [15]:
# pk와 fk 를 설정을 해줬기 때문에 sqlarchemy가 알아서 판단해서 출력해주는것 뿐
# 싱크를 맞추기위해 클래스에게도 알려주는거 
# db에 거치지 않아도 미리 알게 될 수 있게 하는것 

In [3]:
from sqlalchemy.orm import relationship

In [17]:
class Artist(base):
    __tablename__ = "artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    album =relationship("Album", back_populates= "artist")
    
    def __repr__(self):
        return "<T artist(name ='%s')>" % (self.name)

class Album(base):
    __tablename__ = "album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("artist.id"))
    
    artist = relationship("Artist", back_populates = "album", uselist=False)
    
    # 다시 클래스 정의를 해야한다.
    # 1:N 의 관계 리스트 타입이 아니기 때문에 # 꼭 1이여야 한다.  
    # Artist 객체와 관계를 맺고 얘를 추적해서 가면 album(어트리뷰트랑 관련이 있다)을 만날 수 있다 
    
class Genre(base):
    __tablename__ = "genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class Track(base):
    __tablename__ = "track"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    album_id = Column(Integer, ForeignKey('album.id'))
    genre_id = Column(Integer, ForeignKey('genre.id'))

In [20]:
base.metadata.create_all(engine)

2018-07-19 00:24:24,539 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-19 00:24:24,540 INFO sqlalchemy.engine.base.Engine ()
2018-07-19 00:24:24,543 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-19 00:24:24,543 INFO sqlalchemy.engine.base.Engine ()
2018-07-19 00:24:24,545 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-19 00:24:24,546 INFO sqlalchemy.engine.base.Engine ()
2018-07-19 00:24:24,548 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-19 00:24:24,549 INFO sqlalchemy.engine.base.Engine ()


In [21]:
artist1 = Artist(name="Led zepplin")
artist2 = Artist(name="AC/DC")

In [22]:
artist1

<T artist(name ='Led zepplin')>

In [23]:
artist1.album = [Album(title= "IV"), Album(title="who Made Who")]

In [24]:
type(artist1.album[0].artist)

__main__.Artist

In [25]:
from sqlalchemy.orm import sessionmaker

In [26]:
Session = sessionmaker(bind = engine)

In [27]:
session = Session() # con 대신 session

In [28]:
session.add_all([artist1, artist2])

In [29]:
session.commit()

2018-07-19 00:24:30,543 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-19 00:24:30,545 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-19 00:24:30,547 INFO sqlalchemy.engine.base.Engine ('Led zepplin',)
2018-07-19 00:24:30,550 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-19 00:24:30,551 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2018-07-19 00:24:30,553 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-19 00:24:30,554 INFO sqlalchemy.engine.base.Engine ('IV', 7)
2018-07-19 00:24:30,556 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-19 00:24:30,557 INFO sqlalchemy.engine.base.Engine ('who Made Who', 7)
2018-07-19 00:24:30,558 INFO sqlalchemy.engine.base.Engine COMMIT


In [30]:
session.dirty # database 싱크가 맞는지 (내가 알고있는 인스턴슨 값들이 바뀐게 있는지 알려줌 )

IdentitySet([])

In [31]:
print(artist1.id, artist1.name, artist1.album[0].id)

2018-07-19 00:24:31,488 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-19 00:24:31,490 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-19 00:24:31,491 INFO sqlalchemy.engine.base.Engine (7,)
2018-07-19 00:24:31,495 INFO sqlalchemy.engine.base.Engine SELECT album.id AS album_id, album.title AS album_title, album.artist_id AS album_artist_id 
FROM album 
WHERE ? = album.artist_id
2018-07-19 00:24:31,496 INFO sqlalchemy.engine.base.Engine (7,)
7 Led zepplin 9


In [32]:
print(type(artist1)) # list type n:M의 관계라서 

<class '__main__.Artist'>


In [33]:
artist1.album[0].title = "test"

In [34]:
session.dirty

IdentitySet([<__main__.Album object at 0x000002120399D320>])

In [35]:
session.commit() # 커밋을 반드시 해줘야 반영이 된다 .

2018-07-19 00:24:33,465 INFO sqlalchemy.engine.base.Engine UPDATE album SET title=? WHERE album.id = ?
2018-07-19 00:24:33,466 INFO sqlalchemy.engine.base.Engine ('test', 9)
2018-07-19 00:24:33,469 INFO sqlalchemy.engine.base.Engine COMMIT


In [36]:
print(artist1.id, artist1.name, artist1.album[0].id, artist1.album[0].title)

2018-07-19 00:24:33,958 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-19 00:24:33,960 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-19 00:24:33,961 INFO sqlalchemy.engine.base.Engine (7,)
2018-07-19 00:24:33,964 INFO sqlalchemy.engine.base.Engine SELECT album.id AS album_id, album.title AS album_title, album.artist_id AS album_artist_id 
FROM album 
WHERE ? = album.artist_id
2018-07-19 00:24:33,965 INFO sqlalchemy.engine.base.Engine (7,)
7 Led zepplin 9 test


In [37]:
session.new 

IdentitySet([])

In [38]:
artist1.name = "xyz"

In [39]:
session.dirty

IdentitySet([<T artist(name ='xyz')>])

In [40]:
session.commit # 미반영된걸 sql commit

<bound method Session.commit of <sqlalchemy.orm.session.Session object at 0x00000212037EBC88>>

In [41]:
# 매번 인스턴스 만드는게 귀찮고 
# add_all은 무언가 많은걸 넣을 수 있어
# 단지 차이는 우리는 클래스를 사용

In [42]:
session.add_all([    
    Album(title="1", artist_id=1),
    Album(title="2", artist_id=2)
]) # 메모리상에 올라온 것뿐

In [43]:
session.dirty # 바뀜 커밋을 해줘야함

IdentitySet([<T artist(name ='xyz')>])

In [44]:
session.add_all([
    Genre(name="Rock"),
    Genre(name="Metal")
]) # 메모리상에 올라온것뿐

In [45]:
session.add_all([    
    Track(title="1-1", album_id=1, genre_id=1),
    Track(title="1-2", album_id=1, genre_id=2),
    Track(title="2-1", album_id=2, genre_id=1),
    Track(title="2-2", album_id=2, genre_id=2),
]) # 메모리상에 올라온것뿐

In [46]:
session.new

IdentitySet([<__main__.Album object at 0x000002120399DA20>, <__main__.Album object at 0x000002120399DA90>, <__main__.Genre object at 0x000002120399DB70>, <__main__.Genre object at 0x000002120399DBE0>, <__main__.Track object at 0x00000212039A42E8>, <__main__.Track object at 0x00000212039A4358>, <__main__.Track object at 0x00000212039A43C8>, <__main__.Track object at 0x00000212039A4438>])

In [47]:
session.commit # 자동으로 어찌 했는지 쭉 나옴 

<bound method Session.commit of <sqlalchemy.orm.session.Session object at 0x00000212037EBC88>>

In [48]:
temp = Track(title="3-1", album_id=3, genre_id=3)

In [49]:
print(temp.id) # 실제 데이터베이스에 적용을 안했기 때문 

None


In [50]:
session.add(temp)

In [51]:
session.new # 작업이 안되었기 때문에 

IdentitySet([<__main__.Album object at 0x000002120399DA20>, <__main__.Album object at 0x000002120399DA90>, <__main__.Genre object at 0x000002120399DB70>, <__main__.Genre object at 0x000002120399DBE0>, <__main__.Track object at 0x00000212039A42E8>, <__main__.Track object at 0x00000212039A4358>, <__main__.Track object at 0x00000212039A43C8>, <__main__.Track object at 0x00000212039A4438>, <__main__.Track object at 0x000002120399DF28>])

In [52]:
session.commit()

2018-07-19 00:24:41,023 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-19 00:24:41,025 INFO sqlalchemy.engine.base.Engine ('1-1', 1, 1)
2018-07-19 00:24:41,028 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-19 00:24:41,029 INFO sqlalchemy.engine.base.Engine ('1-2', 1, 2)
2018-07-19 00:24:41,030 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-19 00:24:41,031 INFO sqlalchemy.engine.base.Engine ('2-1', 2, 1)
2018-07-19 00:24:41,032 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-19 00:24:41,033 INFO sqlalchemy.engine.base.Engine ('2-2', 2, 2)
2018-07-19 00:24:41,034 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-19 00:24:41,035 INFO sqlalchemy.engine.base.Engine ('3-1', 3, 3)
2018-07-19 00:24:41,037 I

In [53]:
session.new

IdentitySet([])

In [54]:
print(temp.id) # 값을 불러온다 

2018-07-19 00:24:42,252 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-19 00:24:42,254 INFO sqlalchemy.engine.base.Engine SELECT track.id AS track_id, track.title AS track_title, track.album_id AS track_album_id, track.genre_id AS track_genre_id 
FROM track 
WHERE track.id = ?
2018-07-19 00:24:42,255 INFO sqlalchemy.engine.base.Engine (15,)
15


In [55]:
for row in session.query(Track):
    print(row.id, end=" ")
    print(row.album_id)

2018-07-19 00:24:47,953 INFO sqlalchemy.engine.base.Engine SELECT track.id AS track_id, track.title AS track_title, track.album_id AS track_album_id, track.genre_id AS track_genre_id 
FROM track
2018-07-19 00:24:47,954 INFO sqlalchemy.engine.base.Engine ()
1 1
2 1
3 2
4 2
5 3
6 1
7 1
8 2
9 2
10 3
11 1
12 1
13 2
14 2
15 3


In [56]:
for row in session.query(Artist):
    print(row)
    print(row.name)
    print(row.id)

2018-07-19 00:24:53,120 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist
2018-07-19 00:24:53,122 INFO sqlalchemy.engine.base.Engine ()
<T artist(name ='xyz')>
xyz
1
<T artist(name ='AC/DC')>
AC/DC
2
<T artist(name ='Led zepplin')>
Led zepplin
3
<T artist(name ='AC/DC')>
AC/DC
4
<T artist(name ='xyz')>
xyz
5
<T artist(name ='AC/DC')>
AC/DC
6
<T artist(name ='xyz')>
xyz
7
<T artist(name ='AC/DC')>
AC/DC
8


In [57]:
# 메모리상과 디비와 싱크를 맞아야하기 때문에
# 값을 불러올때 메모리상 데이터를 가져온다 (속도 업)

In [58]:
for row in session.query(Artist).filter(Artist.id == 1):
    print(row)
    print(row.name)
    print(row.id)

2018-07-19 00:24:55,379 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-19 00:24:55,381 INFO sqlalchemy.engine.base.Engine (1,)
<T artist(name ='xyz')>
xyz
1


In [64]:
from sqlalchemy import join

In [67]:
result = session.query(Track.title, Album.title, Genre.name, Artist.name).select_from(Track).join(Album).join(Genre).join(Artist).all()

2018-07-19 00:30:12,927 INFO sqlalchemy.engine.base.Engine SELECT track.title AS track_title, album.title AS album_title, genre.name AS genre_name, artist.name AS artist_name 
FROM track JOIN album ON album.id = track.album_id JOIN genre ON genre.id = track.genre_id JOIN artist ON artist.id = album.artist_id
2018-07-19 00:30:12,928 INFO sqlalchemy.engine.base.Engine ()


In [68]:
for row in result:
    print(row)

('1-1', '1', 'Rock', 'xyz')
('1-2', '1', 'Metal', 'xyz')
('2-1', '2', 'Rock', 'AC/DC')
('2-2', '2', 'Metal', 'AC/DC')
('3-1', 'test', 'Rock', 'Led zepplin')
('1-1', '1', 'Rock', 'xyz')
('1-2', '1', 'Metal', 'xyz')
('2-1', '2', 'Rock', 'AC/DC')
('2-2', '2', 'Metal', 'AC/DC')
('3-1', 'test', 'Rock', 'Led zepplin')
('1-1', '1', 'Rock', 'xyz')
('1-2', '1', 'Metal', 'xyz')
('2-1', '2', 'Rock', 'AC/DC')
('2-2', '2', 'Metal', 'AC/DC')
('3-1', 'test', 'Rock', 'Led zepplin')


In [69]:
sonList = [dict(Track=row[0], Album=row[1], Genre=row[2], Artist=row[3]) for row in result]

In [70]:
sonList

[{'Album': '1', 'Artist': 'xyz', 'Genre': 'Rock', 'Track': '1-1'},
 {'Album': '1', 'Artist': 'xyz', 'Genre': 'Metal', 'Track': '1-2'},
 {'Album': '2', 'Artist': 'AC/DC', 'Genre': 'Rock', 'Track': '2-1'},
 {'Album': '2', 'Artist': 'AC/DC', 'Genre': 'Metal', 'Track': '2-2'},
 {'Album': 'test', 'Artist': 'Led zepplin', 'Genre': 'Rock', 'Track': '3-1'},
 {'Album': '1', 'Artist': 'xyz', 'Genre': 'Rock', 'Track': '1-1'},
 {'Album': '1', 'Artist': 'xyz', 'Genre': 'Metal', 'Track': '1-2'},
 {'Album': '2', 'Artist': 'AC/DC', 'Genre': 'Rock', 'Track': '2-1'},
 {'Album': '2', 'Artist': 'AC/DC', 'Genre': 'Metal', 'Track': '2-2'},
 {'Album': 'test', 'Artist': 'Led zepplin', 'Genre': 'Rock', 'Track': '3-1'},
 {'Album': '1', 'Artist': 'xyz', 'Genre': 'Rock', 'Track': '1-1'},
 {'Album': '1', 'Artist': 'xyz', 'Genre': 'Metal', 'Track': '1-2'},
 {'Album': '2', 'Artist': 'AC/DC', 'Genre': 'Rock', 'Track': '2-1'},
 {'Album': '2', 'Artist': 'AC/DC', 'Genre': 'Metal', 'Track': '2-2'},
 {'Album': 'test', 'Ar